In [ ]:
import utils_data as utils

import torch
import torch.nn as nn 
import torch.nn.functional as F
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from torchtext.data.functional import to_map_style_dataset
from torch.utils.data import DataLoader
from torch.utils.data import WeightedRandomSampler
from torch.optim import Adam

from tqdm import tqdm
from sklearn.metrics import accuracy_score
from collections import Counter
import numpy as np
import gc
import math

device = "cuda" if torch.cuda.is_available() else "cpu"

### Importation du dataset

In [ ]:
### Décommentez pour avoir des datasets identiques à chaque test ###

# # Récupération du contenu des différents datasets
# text_train, text_val = utils.get_data_split()
# label_train, label_val = utils.get_labels_split()
# text_test = utils.get_test()

# # Fusion des textes et des labels sous forme d'une liste de tuple
# merged_train = utils.merge_data_labels(text_train, label_train)
# merged_val = utils.merge_data_labels(text_val, label_val)

In [ ]:
### Commentez pour avoir des datasets identiques à chaque test ###

# Solution pour randomiser la séparation train / val
text_all = utils.get_data()
labels_all = utils.get_labels()
limit = int(len(labels_all) * 0.8) + 1

merged_all = utils.merge_data_labels(text_all, labels_all)
np.random.shuffle(merged_all)

merged_train, merged_val = merged_all[:limit], merged_all[limit:] # Séparation entre les données de train et de validation (80% - 20%)

text_test = utils.get_test()

Note importante : toute la suite est inspiré de : https://coderzcolumn.com/tutorials/artificial-intelligence/pytorch-rnn-for-text-classification-tasks, ainsi que https://n8henrie.com/2021/08/writing-a-transformer-classifier-in-pytorch/ et du cours IFT 6135-A2022.

### Création de tokens à partir des phrases

In [ ]:
# On construit notre vocabulaire à partir des mots de la base de données
tokenizer = get_tokenizer("basic_english")

def build_vocabulary(datasets):
    for dataset in datasets:
        for text in dataset:
            yield tokenizer(text)

vocab = build_vocab_from_iterator(build_vocabulary([text_all]), min_freq=1, specials=["<UNK>"])
vocab.set_default_index(vocab["<UNK>"])

### Hyperparamètres

In [ ]:
target_classes = ["negative", "neutral", "positive"]
max_words = 50
embed_len = 50
n_heads = 5
n_layers = 3
p_dropout = 0.1
epochs = 5
learning_rate = 0.001

### Préparation des données

In [ ]:
# Comptage du nombre d'exemples par classe pour établir des poids et un "sampler" (sur l'ensemble des données d'entraînement choisies)
label_train = np.asarray([list(e) for e in merged_train])[:, 1].astype(int)
counter = Counter(label_train.tolist())

class_weights = [len(label_train) / counter[i] for i in range(len(counter))]
weights = [class_weights[label_train[i]] for i in range(len(label_train))]
sampler = WeightedRandomSampler(torch.DoubleTensor(weights), len(label_train))

In [ ]:
# Préparation des données sous un format lisible par PyTorch
data_train, data_val = to_map_style_dataset(merged_train), to_map_style_dataset(merged_val)
data_test = to_map_style_dataset(text_test)

# Fonction servant à transformer nos données (textes) sous formes de nombres.
def vectorize_batch(batch):
    X, Y = list(zip(*batch))
    X = [vocab(tokenizer(text)) for text in X]
    X = [tokens+([0]* (max_words-len(tokens))) if len(tokens)<max_words else tokens[:max_words] for tokens in X] ## Bringing all samples to max_words length.

    return torch.tensor(X, dtype=torch.int32), torch.tensor(Y, dtype = torch.long)

# Fonction servant à transformer nos données (textes) sous formes de nombres (dans le cas de données de test, on a pas de labels)
def vectorize_test_batch(batch):
    X = list(batch)
    X = [vocab(tokenizer(text)) for text in X]
    X = [tokens+([0]* (max_words-len(tokens))) if len(tokens)<max_words else tokens[:max_words] for tokens in X] ## Bringing all samples to max_words length.

    return torch.tensor(X, dtype=torch.int32)

# Préparation des DataLoader pouvant être lu dans les modèles PyTorch
train_loader = DataLoader(data_train, batch_size=512, collate_fn=vectorize_batch, sampler=sampler)
val_loader = DataLoader(data_val, batch_size=512, collate_fn=vectorize_batch)
test_loader = DataLoader(data_test, batch_size=512, collate_fn=vectorize_test_batch)

### Définition du Transformer

In [ ]:
# Classe permettant d'encoder le postionnement des mots dans le texte (pour que le Transformer en tienne compte)
class PositionalEncoding(nn.Module):
    """
    https://pytorch.org/tutorials/beginner/transformer_tutorial.html
    """

    def __init__(self, d_model, vocab_size=5000, dropout=0.1):
        super().__init__()
        self.dropout = nn.Dropout(p=dropout)

        pe = torch.zeros(vocab_size, d_model)
        position = torch.arange(0, vocab_size, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(
            torch.arange(0, d_model, 2).float()
            * (-math.log(10000.0) / d_model)
        )
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0)
        self.register_buffer("pe", pe)

    def forward(self, x):
        x = x + self.pe[:, : x.size(1), :]
        return self.dropout(x)

In [ ]:
# Architecture de notre Transformer
class Transformer(nn.Module):
  def __init__(self, nhead=n_heads, dim_feedforward=2048, num_layers=n_layers, dropout=p_dropout, activation="relu", classifier_dropout=p_dropout):

    super().__init__()
    self.d_model = embed_len

    assert self.d_model % nhead == 0, "nheads must divide evenly into d_model"
    self.emb = nn.Embedding(num_embeddings=len(vocab), embedding_dim=self.d_model)
    self.pos_encoder = PositionalEncoding(d_model=self.d_model, dropout=dropout, vocab_size=len(vocab))
    encoder_layer = nn.TransformerEncoderLayer(d_model=self.d_model, nhead=nhead, dim_feedforward=dim_feedforward, dropout=dropout)
    self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)
    self.classifier = nn.Linear(self.d_model, len(target_classes))


  def forward(self, x):
    x = self.emb(x) * math.sqrt(self.d_model)
    x = self.pos_encoder(x)
    x = self.transformer_encoder(x)
    x = x.mean(dim=1)
    x = F.softmax(self.classifier(x), dim=-1)

    return x

### Entraînement du Transformer

In [ ]:
# Fonction d'évaluation du RNN
def evaluate(model, loss_fn, val_loader):
    model.eval()
    with torch.no_grad():
        Y_true, Y_preds, losses = [],[],[]
        for X, Y in val_loader:
            Y_true.append(Y)
            X, Y = X.to(device), Y.to(device)
            preds = model(X)
            loss = loss_fn(preds, Y)
            losses.append(loss.item())
            Y_preds.append(preds.argmax(dim=-1))

        Y_true = torch.cat(Y_true)
        Y_preds = torch.cat(Y_preds)

        print("Valid Loss : {:.3f} | Valid Acc : {:.3f}".format(torch.tensor(losses).mean(), accuracy_score(Y_true.detach().cpu().numpy(), Y_preds.detach().cpu().numpy())))

# Fonction d'entraînement du Transformer
def train(model, loss_fn, optimizer, train_loader, val_loader, epochs=10):
    for i in range(1, epochs+1):
        Y_true, Y_preds, losses = [],[],[]
        for X, Y in tqdm(train_loader):
            Y_true.append(Y)
            X, Y = X.to(device), Y.to(device)
            optimizer.zero_grad()
            preds = model(X)
            loss = loss_fn(preds, Y)
            losses.append(loss.item())
            Y_preds.append(preds.argmax(dim=-1))
            loss.backward()
            optimizer.step()

        Y_true = torch.cat(Y_true)
        Y_preds = torch.cat(Y_preds)
            
        print("Epoch {} | Train Loss : {:.3f} | Train acc : {:.3f}".format(i, torch.tensor(losses).mean(), accuracy_score(Y_true.detach().cpu().numpy(), Y_preds.detach().cpu().numpy())))
        evaluate(model, loss_fn, val_loader)
        model.train()

In [ ]:
# On déclare le modèle, la fonction de coût et l'optimiseur avant de lancer l'entraînement
loss_fn = nn.CrossEntropyLoss()
transformer = Transformer().to(device)
optimizer = Adam(transformer.parameters(), lr=learning_rate)

train(transformer, loss_fn, optimizer, train_loader, val_loader, epochs)

### Prédictions sur l'ensemble de test

In [ ]:
# Fonction de prédiction sur un ensemble de test
def predict(model, loader):
    Y_preds = []
    for X in tqdm(loader):
        X = X.to(device)
        preds = model(X)
        Y_preds.append(preds.detach().cpu())
    gc.collect()
    Y_preds = torch.cat(Y_preds)

    return Y_preds.argmax(dim=-1).numpy()

Y_preds = predict(transformer, test_loader)

### Enregistrement des résultats

In [ ]:
utils.save_results(Y_preds, "Transformer")